# LiDAR dataset
I am using OpenTopography for Peoria, AZ 
- https://medium.com/spatial-data-science/what-is-lidar-point-cloud-data-a547ed29edf5

In [1]:
import laspy
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Loading and Exploring Data

In [2]:
# import lidar .las data and assign to variable
las = laspy.read("data/points.las")

In [3]:
las

<LasData(1.0, point fmt: <PointFormat(1, 0 bytes of extra dims)>, 156545175 points, 3 vlrs)>

In [4]:
las.header
las.header.point_format
las.header.point_count
las.vlrs

[<GeoKeyDirectoryVlr(11 geo_keys)>, <GeoAsciiParamsVlr(['NAD83 / UTM zone 12N + VERT_CS|NAD83|NAVD88 height|'])>, <VLR(user_id: 'liblas', record_id: '2112', data len: 951)>]

In [5]:
# examine the available features for the lidar file we have read
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time']

In [6]:
print("X")
print(las.X)
print("Intensity")
print(las.intensity)
print("GPS time")
print(las.gps_time)
print("Key Point")
print(las.key_point)

X
[36800000 36800000 36800001 ... 36904564 36904408 36904407]
Intensity
[37 47 48 ... 63 69 66]
GPS time
[491938.793022 491938.79303  491938.793038 ... 499842.311392 499842.333713
 499842.333721]
Key Point
<SubFieldView([0 0 0 ... 0 0 0])>


In [7]:
set(list(las.classification))

{1, 2, 6}

Class Labels
- 1 = Unassigned
- 2 = Ground
- 5 = High Vegetation
- 6 = Building

# Creating, Filtering, and Writing Point Cloud Data

In [8]:
# To create 3D point cloud data, we can stack together with the X, Y, and Z dimensions, using Numpy like this.
# point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))
point_data = np.vstack((las.X, las.Y, las.Z)).transpose()

point_data.shape

(156545175, 3)

In [9]:
las.header.version

Version(major=1, minor=0)

In [10]:
buildings = laspy.create(point_format=las.header.point_format)
buildings.points = las.points[las.classification == 6]

In [11]:
buildings.write('buildings.las')

# 3D Point Cloud Visualization

In [12]:
# Laspy has no visualization methods so that we will use the open3d library. We first create the open3D geometries and pass the point data we have created earlier. Finally, we use the open3d visualization to draw geometries.
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
voxel_pcd = geom.points .voxel_down_sample(voxel_size=0.1)
o3d.visualization.draw_geometries([geom])

AttributeError: 'open3d.cuda.pybind.utility.Vector3dVector' object has no attribute 'voxel_down_sample'